In [ ]:
pip install -q tf-nightly

In [ ]:
!pip install tensorflow-io

In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
from pathlib import Path
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint
from tensorflow.keras.preprocessing import image,image_dataset_from_directory
import matplotlib.pyplot as plt
import tensorflow_io as tfio
tf.random.set_seed(0)

In [ ]:
os.environ['TF_AZURE_STORAGE_KEY'] = 'Azure Storage Key'
account_name = 'Azure-based-inventory-management-app'

In [ ]:
pathname = 'az://{}/toronto-tagging-photo/'.format(account_name)
tf.io.gfile.listdir(
    pathname
)

tf.io.gfile.mkdir(pathname)
filename = pathname + '/000587-B22-B2034-Overhead Doors-91005723-Sys.png'

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
PATH = "/content/drive/My Drive/CBRE_Training_Images/Structure"
datadir = Path(PATH)
%cd /content/drive/MyDrive/CBRE_Training_Images

In [ ]:
train_dir = os.path.join(datadir, 'train')
validation_dir = os.path.join(datadir, 'val')

In [ ]:
image_generator = ImageDataGenerator(rescale=1./255) # Generator for our data

In [ ]:
batch_size = 16
IMG_HEIGHT = 160
IMG_WIDTH = 160
epochs=25
IMG_SHAPE =(IMG_HEIGHT,IMG_WIDTH,3)

In [ ]:
train_data_gen = image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='categorical')

val_data_gen = image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='categorical')                                                        

In [ ]:
num_classes = train_data_gen.num_classes

In [ ]:
base_model = InceptionV3(weights= 'imagenet', include_top=False, input_shape= IMG_SHAPE)

# Freeze layers in the base model (i.e. only train the classifier)
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation= 'relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

In [ ]:
model.compile(optimizer= 'Adam', loss='categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

In [ ]:
checkpoint_cb = ModelCheckpoint("CNN2_Cat", save_best_only=True)
history = model.fit(
    train_data_gen,
    callbacks =checkpoint_cb,
    steps_per_epoch=train_data_gen.n // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=val_data_gen.n// batch_size
)

In [ ]:
CNN2 = keras.models.load_model("CNN2_Cat")

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(epochs)
acc = np.array(acc)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1,6, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

# class_names = train_dataset.class_names

# plt.figure(figsize=(10, 10))
# for images, labels in train_dataset.take(1):
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(class_names[labels[i]])
#     plt.axis("off")

In [ ]:
sample_training_images, _ = next(train_data_gen)
plotImages(sample_training_images)

In [ ]:
sample_predictions = CNN2.predict(sample_training_images)
sample_predictions

In [ ]:
sample_training_images2, _ = next(train_data_gen)
plotImages(sample_training_images2)
sample_predictions2 = CNN2.predict(sample_training_images2)
sample_predictions2

In [ ]:
test_image = image.load_img(filename)
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)
test_image
# predictions = CNN2.predict(test_image)

# predictions